In [ ]:
import numpy as np
import os
import io
import PIL
import PIL.Image
import PIL.ImageOps
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
from PIL import ImageOps
from PIL import ImageColor
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from keras.layers import *
import time
import tempfile
from urllib.request import urlopen
from io import BytesIO
from keras.preprocessing import image
import numpy as np 
import urllib
import cv2 
import os

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory("DatasetOOT/Training/",
                                      batch_size=10, shuffle = True, image_size=(300,300))

In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory("DatasetOOT/Validation/",
                                                          batch_size=4, image_size=(300,300))

In [ ]:
train_dataset.class_names
class_names = train_dataset.class_names
print(class_names)

In [ ]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal",input_shape=(300,300,3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomZoom(-0.1),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.GaussianNoise(2),
    tf.keras.layers.GaussianNoise(1),
    tf.keras.layers.RandomBrightness([-0.2, 0.5])
  ]
)

best_callbacks = [tf.keras.callbacks.EarlyStopping(patience = 1000) ,
                 tf.keras.callbacks.ModelCheckpoint(filepath="OOTTrainedModel/",monitor="val_accuracy",
                    save_best_only=True,
                    save_weights_only=False,
                    mode="auto",

In [ ]:
def build_model(input_shape):
    inputs = Input(input_shape)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(4, activation="softmax")(x)
    model = tf.keras.models.Model(inputs, x)
    return model

In [ ]:
def build_model(input_shape):
    inputs = Input(input_shape)
    inputs = data_augmentation(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(2, activation="softmax")(x)
    model = tf.keras.models.Model(inputs, x)
    return model

In [ ]:
model = build_model((300,300, 3))

In [ ]:
model.fit = model.fit(train_dataset,epochs=10, 
                      validation_data=validation_dataset, 
                      shuffle=True)

In [ ]:
model.summary()